# CatBoost Starter for Brain Comp
EEGの差分の集計特徴量を用いたNotebook。

- ver1: CV = 1.26,
- ver2: CV = 1.175, LB = 0.93
- ver3: CV = 1.168, LB = 0.98
- ver4: CV = 0.798, LB = 0.63

### Version Notes
- Version 1 - EEGのT=20, 30, 40秒における、10秒時間窓 / 20秒時間窓のmean, min, max, std, max-minを特徴量とした。また、データの1/20のみ利用。
- Version 2 - EEGのT=20, 30秒における、10秒時間窓 / 20秒時間窓のmean, min, max, std, max-minを特徴量とした。時間窓に必要な部分のみ計算することで高速化した。
- Version 3 - EEGのT=10, 20, 25, 30, 40秒における、5秒時間窓 / 10秒時間窓のmean, min, max, std, max-minを特徴量とした。差分をとる電極を追加した。
- Version 4 - 公開notebookの、スペクトログラムによる特徴量を追加した。

# Load Libraries

In [44]:
import os, gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import time
pl.Config.set_tbl_cols(-1)

VER = 4

# Load Train Data

In [45]:
df_train = pl.read_csv('../input/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df_train.columns[-6:]
print('Train shape:', df_train.shape )
print('Targets', list(TARGETS))
df_train.head()

Train shape: (106800, 15)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
i64,i64,f64,i64,i64,f64,i64,i64,str,i64,i64,i64,i64,i64,i64
1628180742,0,0.0,353733,0,0.0,127492639,42516,"""Seizure""",3,0,0,0,0,0
1628180742,1,6.0,353733,1,6.0,3887563113,42516,"""Seizure""",3,0,0,0,0,0
1628180742,2,8.0,353733,2,8.0,1142670488,42516,"""Seizure""",3,0,0,0,0,0
1628180742,3,18.0,353733,3,18.0,2718991173,42516,"""Seizure""",3,0,0,0,0,0
1628180742,4,24.0,353733,4,24.0,3080632009,42516,"""Seizure""",3,0,0,0,0,0


# Create Multiple Eeg Id Train Data
データの説明[here][1]には、テストデータには、同じ `eeg_id` からの複数の crop は含まれていないと記載されている。<br>
ひとつの`eeg_id`ごとのデータにする

[1]: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467021

In [46]:
df_train = df_train.group_by(by = 'eeg_id').agg(
    pl.first("spectrogram_id").alias("spec_id"),
    pl.min("eeg_label_offset_seconds").alias("eeg_min_offset"),
    pl.max("eeg_label_offset_seconds").alias("eeg_max_offset"),
    pl.first("patient_id"),
    pl.sum("seizure_vote"),
    pl.sum("lpd_vote"),
    pl.sum("gpd_vote"),
    pl.sum("lrda_vote"),
    pl.sum("grda_vote"),
    pl.sum("other_vote"),
    pl.first("expert_consensus")
)
df_train.head()


# 各vote列の合計
df_train = df_train.with_columns(
    (pl.col('seizure_vote') + pl.col('lpd_vote') + pl.col('gpd_vote') + pl.col('lrda_vote') + pl.col('grda_vote') + pl.col('other_vote')).alias('vote_sum')
)

# 各vote列の正規化
df_train = df_train.with_columns(
    (pl.col('seizure_vote') / pl.col('vote_sum')).alias('seizure_vote'),
    (pl.col('lpd_vote') / pl.col('vote_sum')).alias('lpd_vote'),
    (pl.col('gpd_vote') / pl.col('vote_sum')).alias('gpd_vote'),
    (pl.col('lrda_vote') / pl.col('vote_sum')).alias('lrda_vote'),
    (pl.col('grda_vote') / pl.col('vote_sum')).alias('grda_vote'),
    (pl.col('other_vote') / pl.col('vote_sum')).alias('other_vote'),
)

# vote列の削除
df_train = df_train.drop(["vote_sum"])

# expert_consensus列をtarget列とする
df_train = df_train.rename({"expert_consensus": "target"})
df_train.head()

eeg_id,spec_id,eeg_min_offset,eeg_max_offset,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
i64,i64,f64,f64,i64,f64,f64,f64,f64,f64,f64,str
1604920026,1338193223,0.0,100.0,60066,0.0,0.0,0.0,0.0,0.0,1.0,"""Other"""
990449207,2126560277,0.0,6.0,22597,0.818182,0.0,0.181818,0.0,0.0,0.0,"""Seizure"""
3957673122,44306235,0.0,0.0,9969,0.0,0.0,0.0,0.0,0.0,1.0,"""Other"""
498482803,107927759,0.0,2.0,13133,0.0,0.2,0.05,0.0,0.0,0.75,"""Other"""
1235456176,174063666,0.0,10.0,13732,1.0,0.0,0.0,0.0,0.0,0.0,"""Seizure"""


# Feature Engineer


CatBoostモデルに入れる特徴量を作成する。


## 1. spectrogram
KaggleスペクトログラムとEEGスペクトログラムの両方を使用する特徴量

In [47]:
train = df_train.to_pandas()

In [48]:
# Falseのとき、CHRIS DEOTTE氏が作成したデータを利用
READ_SPEC_FILES = False
READ_EEG_SPEC_FILES = False

In [49]:
%%time
# READ ALL SPECTROGRAMS
# スペクトログラムの読み込み
PATH = '../input/hms-harmful-brain-activity-classification/train_spectrograms/'
files = os.listdir(PATH)
print(f'There are {len(files)} spectrogram parquets')

if READ_SPEC_FILES:    
    spectrograms = {}
    for i,f in enumerate(files):
        if i%100==0: print(i,', ',end='')
        tmp = pd.read_parquet(f'{PATH}{f}')
        name = int(f.split('.')[0])
        spectrograms[name] = tmp.iloc[:,1:].values
else:
    spectrograms = np.load('../input/brain-spectrograms/specs.npy',allow_pickle=True).item()

There are 11138 spectrogram parquets
CPU times: total: 0 ns
Wall time: 6.68 s


In [50]:
%%time
# READ ALL EEG SPECTROGRAMS
# 生のEEGデータから作成されたEEG スペクトログラムの読み込み
if READ_EEG_SPEC_FILES:
    all_eegs = {}
    for i,e in enumerate(train.eeg_id.values):
        if i%100==0: print(i,', ',end='')
        x = np.load(f'../input/brain-eeg-spectrograms/EEG_Spectrograms/{e}.npy')
        all_eegs[e] = x
else:
    all_eegs = np.load('../input/brain-eeg-spectrograms/eeg_specs.npy',allow_pickle=True).item()

CPU times: total: 203 ms
Wall time: 19.2 s


In [51]:
%time
# ENGINEER FEATURES
import warnings
warnings.filterwarnings('ignore')

# FEATURE NAMES
# 元の Spectrogram のファイルから、列名を取得
SPEC_COLS = pd.read_parquet(f'{PATH}1000086677.parquet').columns[1:]
FEATURES = [f'{c}_mean_10m' for c in SPEC_COLS]             # スペクトログラム: 10分の時間窓の時間平均
FEATURES += [f'{c}_min_10m' for c in SPEC_COLS]             # スペクトログラム: 10分の時間窓の時間最小値
FEATURES += [f'{c}_mean_20s' for c in SPEC_COLS]            # スペクトログラム: 20秒の時間窓の時間平均
FEATURES += [f'{c}_min_20s' for c in SPEC_COLS]             # スペクトログラム: 20秒の時間窓の時間最小値
FEATURES += [f'eeg_mean_f{x}_10s' for x in range(512)]      # 脳波: 10分の時間窓の時間平均
FEATURES += [f'eeg_min_f{x}_10s' for x in range(512)]       # 脳波: 10分の時間窓の時間最小値
FEATURES += [f'eeg_max_f{x}_10s' for x in range(512)]       # 脳波: 20秒の時間窓の時間平均
FEATURES += [f'eeg_std_f{x}_10s' for x in range(512)]       # 脳波: 20秒の時間窓の時間平均
print(f'We are creating {len(FEATURES)} features for {len(train)} rows... ',end='')

data = np.zeros((len(train),len(FEATURES)))
for k in range(len(train)):
    if k%100==0: print(k,', ',end='')
    row = train.iloc[k]
    r = int( (row['eeg_min_offset'] + row['eeg_min_offset'])//4 ) 
    # if k % 10 != 0: continue
    ### スペクトログラム特徴量の計算
    # 10 MINUTE WINDOW FEATURES (MEANS and MINS)
    # 10分時間窓の特徴量計算
    x = np.nanmean(spectrograms[row.spec_id][r:r+300,:],axis=0)
    data[k,:400] = x
    x = np.nanmin(spectrograms[row.spec_id][r:r+300,:],axis=0)
    data[k,400:800] = x

    # 20 SECOND WINDOW FEATURES (MEANS and MINS)
    # 20秒時間窓の特徴量計算
    x = np.nanmean(spectrograms[row.spec_id][r+145:r+155,:],axis=0)
    data[k,800:1200] = x
    x = np.nanmin(spectrograms[row.spec_id][r+145:r+155,:],axis=0)
    data[k,1200:1600] = x

    ### EEG特徴量の計算
    # RESHAPE EEG SPECTROGRAMS 128x256x4 => 512x256
    eeg_spec = np.zeros((512,256),dtype='float32')
    xx = all_eegs[row.eeg_id]
    for j in range(4): eeg_spec[128*j:128*(j+1),] = xx[:,:,j]

    # 10 SECOND WINDOW FROM EEG SPECTROGRAMS 
    # 10秒時間窓の特徴量の計算（mean, min, max, std）
    x = np.nanmean(eeg_spec.T[100:-100,:],axis=0)
    data[k,1600:2112] = x
    x = np.nanmin(eeg_spec.T[100:-100,:],axis=0)
    data[k,2112:2624] = x
    x = np.nanmax(eeg_spec.T[100:-100,:],axis=0)
    data[k,2624:3136] = x
    x = np.nanstd(eeg_spec.T[100:-100,:],axis=0)
    data[k,3136:3648] = x

train[FEATURES] = data
print(); print('New train shape:',train.shape)

CPU times: total: 0 ns
Wall time: 0 ns


We are creating 3648 features for 17089 rows... 0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , 3900 , 4000 , 4100 , 4200 , 4300 , 4400 , 4500 , 4600 , 4700 , 4800 , 4900 , 5000 , 5100 , 5200 , 5300 , 5400 , 5500 , 5600 , 5700 , 5800 , 5900 , 6000 , 6100 , 6200 , 6300 , 6400 , 6500 , 6600 , 6700 , 6800 , 6900 , 7000 , 7100 , 7200 , 7300 , 7400 , 7500 , 7600 , 7700 , 7800 , 7900 , 8000 , 8100 , 8200 , 8300 , 8400 , 8500 , 8600 , 8700 , 8800 , 8900 , 9000 , 9100 , 9200 , 9300 , 9400 , 9500 , 9600 , 9700 , 9800 , 9900 , 10000 , 10100 , 10200 , 10300 , 10400 , 10500 , 10600 , 10700 , 10800 , 10900 , 11000 , 11100 , 11200 , 11300 , 11400 , 11500 , 11600 , 11700 , 11800 , 11900 , 12000 , 12100 , 12200 , 12300 , 12400 , 12500 , 12600 , 12700 , 12800 , 12900 , 13000 , 13100 , 13200 , 

In [52]:
train.head()

,eeg_id,spec_id,eeg_min_offset,eeg_max_offset,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,...,eeg_std_f502_10s,eeg_std_f503_10s,eeg_std_f504_10s,eeg_std_f505_10s,eeg_std_f506_10s,eeg_std_f507_10s,eeg_std_f508_10s,eeg_std_f509_10s,eeg_std_f510_10s,eeg_std_f511_10s
0,1604920026,1338193223,0.0,100.0,60066,0.000000,0.0,0.000000,0.0,0.0,...,0.066399,0.069853,0.082195,0.093638,0.083926,0.081355,0.07971,0.077772,0.077243,0.072499
1,990449207,2126560277,0.0,6.0,22597,0.818182,0.0,0.181818,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,3957673122,44306235,0.0,0.0,9969,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
3,498482803,107927759,0.0,2.0,13133,0.000000,0.2,0.050000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
4,1235456176,174063666,0.0,10.0,13732,1.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


In [53]:
train_spec = train.drop(["spec_id", "eeg_min_offset", "eeg_max_offset", "patient_id",
                         "seizure_vote", "lpd_vote", "gpd_vote", "lrda_vote", "grda_vote", "other_vote",
                         "target"], axis = 1)
train_spec.shape

(17089, 3649)

In [54]:
# FREE MEMORY
del all_eegs, spectrograms, train
gc.collect()

0

## 2. 生のEEG
生のEEGの集計特徴量

In [55]:
# parameter
# 差分特徴量の集計特徴量をつくる
# window = 4000 # 20秒窓 (20秒 / (1レコード1/200秒) = 4000レコード)
center_seconds_list = [10, 10, 20, 20, 25, 25, 30, 30, 40, 40]
window_list         = [1000, 2000, 1000, 2000, 1000, 2000, 1000, 2000, 1000, 2000]

In [56]:
def create_aggregated_features_on_EEG_differences(df_eeg_diff, window, center_sec):
    """
    @Args
    df_eeg_diff : 差分特徴量を持つデータフレーム
    window      : 時間窓として抽出する行数 [行]
    center_sec  : 時間窓の中心時刻 [s]
    
    @Returns
    df_features    : 集計特徴量を持つデータフレーム
    """
    ### 時間窓の設定
    window_time = window / (200)
    str_window = str(int(window_time)) + "s"

    # 時間窓 [window_min, window_max] に入るレコードのみ抽出
    window_max = center_sec + (window / 2)
    window_min = center_sec - (window / 2)

    df_eeg_diff = df_eeg_diff.filter((pl.col("seconds_eeg") <= window_max) & (pl.col("seconds_eeg") >= window_min))
    df_eeg_diff = df_eeg_diff.drop(['seconds_eeg'])

    
    ### 特徴量作成
    
    df_features = df_eeg_diff.group_by(by = "eeg_id").agg(
        ### 簡易特徴量
        # LL: Fp1 - T3
        pl.col("LL_Fp1-T3").mean().alias("mean_" + str_window + "_LL_Fp1-T3"),
        pl.col("LL_Fp1-T3").max().alias("min_" + str_window + "_LL_Fp1-T3"),
        pl.col("LL_Fp1-T3").min().alias("max_" + str_window + "_LL_Fp1-T3"),
        pl.col("LL_Fp1-T3").std().alias("std_" + str_window + "_LL_Fp1-T3"),
        (pl.col("LL_Fp1-T3").max() - pl.col("LL_Fp1-T3").min()).alias("max-min_" + str_window + "_LL_Fp1-T3"),

        # LL: Fp1 - T3
        pl.col("LL_T3-O1").mean().alias("mean_" + str_window + "_LL_T3-O1"),
        pl.col("LL_T3-O1").max().alias("min_" + str_window + "_LL_T3-O1"),
        pl.col("LL_T3-O1").min().alias("max_" + str_window + "_LL_T3-O1"),
        pl.col("LL_T3-O1").std().alias("std_" + str_window + "_LL_T3-O1"),
        (pl.col("LL_T3-O1").max() - pl.col("LL_T3-O1").min()).alias("max-min__" + str_window + "_LL_T3-O1"),


        # LP: Fp1 - C3
        pl.col("LP_Fp1-C3").mean().alias("mean_" + str_window + "_LP_Fp1-C3"),
        pl.col("LP_Fp1-C3").max().alias("min_" + str_window + "_LP_Fp1-C3"),
        pl.col("LP_Fp1-C3").min().alias("max_" + str_window + "_LP_Fp1-C3"),
        pl.col("LP_Fp1-C3").std().alias("std_" + str_window + "_LP_Fp1-C3"),
        (pl.col("LP_Fp1-C3").max() - pl.col("LP_Fp1-C3").min()).alias("max-min_" + str_window + "_LP_Fp1-C3"),

        # LP: C3 - O1
        pl.col("LP_C3-O1").mean().alias("mean_" + str_window + "_LP_C3-O1"),
        pl.col("LP_C3-O1").max().alias("min_" + str_window + "_LP_C3-O1"),
        pl.col("LP_C3-O1").min().alias("max_" + str_window + "_LP_C3-O1"),
        pl.col("LP_C3-O1").std().alias("std_" + str_window + "_LP_C3-O1"),
        (pl.col("LP_C3-O1").max() - pl.col("LP_C3-O1").min()).alias("max-min_" + str_window + "_LP_C3-O1"),


        # RP: Fp1 - C4
        pl.col("RP_Fp2-C4").mean().alias("mean_" + str_window + "_RP_Fp2-C4"),
        pl.col("RP_Fp2-C4").max().alias("min_" + str_window + "_RP_Fp2-C4"),
        pl.col("RP_Fp2-C4").min().alias("max_" + str_window + "_RP_Fp2-C4"),
        pl.col("RP_Fp2-C4").std().alias("std_" + str_window + "_RP_Fp2-C4"),
        (pl.col("RP_Fp2-C4").max() - pl.col("RP_Fp2-C4").min()).alias("max-min_" + str_window + "_RP_Fp2-C4"),

        # RP: C4 - O2
        pl.col("RP_C4-O2").mean().alias("mean_" + str_window + "_RP_C4-O2"),
        pl.col("RP_C4-O2").max().alias("min_" + str_window + "_RP_C4-O2"),
        pl.col("RP_C4-O2").min().alias("max_" + str_window + "_RP_C4-O2"),
        pl.col("RP_C4-O2").std().alias("std_" + str_window + "_RP_C4-O2"),
        (pl.col("RP_C4-O2").max() - pl.col("RP_C4-O2").min()).alias("max-min_" + str_window + "_RP_C4-O2"),


        # RR: Fp2 - T4
        pl.col("RR_Fp2-T4").mean().alias("mean_" + str_window + "_RR_Fp2-T4"),
        pl.col("RR_Fp2-T4").max().alias("min_" + str_window + "_RR_Fp2-T4"),
        pl.col("RR_Fp2-T4").min().alias("max_" + str_window + "_RR_Fp2-T4"),
        pl.col("RR_Fp2-T4").std().alias("std_" + str_window + "_RR_Fp2-T4"),
        (pl.col("RR_Fp2-T4").max() - pl.col("RR_Fp2-T4").min()).alias("max-min_" + str_window + "_RR_Fp2-T4"),

        # RR: T4 - O2
        pl.col("RR_T4-O2").mean().alias("mean_" + str_window + "_RR_T4-O2"),
        pl.col("RR_T4-O2").max().alias("min_" + str_window + "_RR_T4-O2"),
        pl.col("RR_T4-O2").min().alias("max_" + str_window + "_RR_T4-O2"),
        pl.col("RR_T4-O2").std().alias("std_" + str_window + "_RR_T4-O2"),
        (pl.col("RR_T4-O2").max() - pl.col("RR_T4-O2").min()).alias("max-min_" + str_window + "_RR_T4-O2"),

        
        ### 全差分の特徴量
        # LL: Fp1 - F7
        pl.col("LL_Fp1-F7").mean().alias("mean_" + str_window + "_LL_Fp1-F7"),
        pl.col("LL_Fp1-F7").max().alias("min_" + str_window + "_LL_Fp1-F7"),
        pl.col("LL_Fp1-F7").min().alias("max_" + str_window + "_LL_Fp1-F7"),
        pl.col("LL_Fp1-F7").std().alias("std_" + str_window + "_LL_Fp1-F7"),
        (pl.col("LL_Fp1-F7").max() - pl.col("LL_Fp1-F7").min()).alias("max-min_" + str_window + "_LL_Fp1-F7"),

        # LL: F7 - T3
        pl.col("LL_F7-T3").mean().alias("mean_" + str_window + "_LL_F7-T3"),
        pl.col("LL_F7-T3").max().alias("min_" + str_window + "_LL_F7-T3"),
        pl.col("LL_F7-T3").min().alias("max_" + str_window + "_LL_F7-T3"),
        pl.col("LL_F7-T3").std().alias("std_" + str_window + "_LL_F7-T3"),
        (pl.col("LL_F7-T3").max() - pl.col("LL_F7-T3").min()).alias("max-min_" + str_window + "_LL_F7-T3"),

        # LL: T3 - T5
        pl.col("LL_T3-T5").mean().alias("mean_" + str_window + "_LL_T3-T5"),
        pl.col("LL_T3-T5").max().alias("min_" + str_window + "_LL_T3-T5"),
        pl.col("LL_T3-T5").min().alias("max_" + str_window + "_LL_T3-T5"),
        pl.col("LL_T3-T5").std().alias("std_" + str_window + "_LL_T3-T5"),
        (pl.col("LL_T3-T5").max() - pl.col("LL_T3-T5").min()).alias("max-min_" + str_window + "_LL_T3-T5"),

        # LL: T5 - O1
        pl.col("LL_T5-O1").mean().alias("mean_" + str_window + "_LL_T5-O1"),
        pl.col("LL_T5-O1").max().alias("min_" + str_window + "_LL_T5-O1"),
        pl.col("LL_T5-O1").min().alias("max_" + str_window + "_LL_T5-O1"),
        pl.col("LL_T5-O1").std().alias("std_" + str_window + "_LL_T5-O1"),
        (pl.col("LL_T5-O1").max() - pl.col("LL_T5-O1").min()).alias("max-min_" + str_window + "_LL_T5-O1"),



        # LP: Fp1 - F3
        pl.col("LP_Fp1-F3").mean().alias("mean_" + str_window + "_LP_Fp1-F3"),
        pl.col("LP_Fp1-F3").max().alias("min_" + str_window + "_LP_Fp1-F3"),
        pl.col("LP_Fp1-F3").min().alias("max_" + str_window + "_LP_Fp1-F3"),
        pl.col("LP_Fp1-F3").std().alias("std_" + str_window + "_LP_Fp1-F3"),
        (pl.col("LP_Fp1-F3").max() - pl.col("LP_Fp1-F3").min()).alias("max-min_" + str_window + "_LP_Fp1-F3"),

        # LP: F3 - C3
        pl.col("LP_F3-C3").mean().alias("mean_" + str_window + "_LP_F3-C3"),
        pl.col("LP_F3-C3").max().alias("min_" + str_window + "_LP_F3-C3"),
        pl.col("LP_F3-C3").min().alias("max_" + str_window + "_LP_F3-C3"),
        pl.col("LP_F3-C3").std().alias("std_" + str_window + "_LP_F3-C3"),
        (pl.col("LP_F3-C3").max() - pl.col("LP_F3-C3").min()).alias("max-min_" + str_window + "_LP_F3-C3"),

        # LP: C3 - P3
        pl.col("LP_C3-P3").mean().alias("mean_" + str_window + "_LP_C3-P3"),
        pl.col("LP_C3-P3").max().alias("min_" + str_window + "_LP_C3-P3"),
        pl.col("LP_C3-P3").min().alias("max_" + str_window + "_LP_C3-P3"),
        pl.col("LP_C3-P3").std().alias("std_" + str_window + "_LP_C3-P3"),
        (pl.col("LP_C3-P3").max() - pl.col("LP_C3-P3").min()).alias("max-min_" + str_window + "_LP_C3-P3"),

        # LP: P3 - O1
        pl.col("LP_P3-O1").mean().alias("mean_" + str_window + "_LP_P3-O1"),
        pl.col("LP_P3-O1").max().alias("min_" + str_window + "_LP_P3-O1"),
        pl.col("LP_P3-O1").min().alias("max_" + str_window + "_LP_P3-O1"),
        pl.col("LP_P3-O1").std().alias("std_" + str_window + "_LP_P3-O1"),
        (pl.col("LP_P3-O1").max() - pl.col("LP_P3-O1").min()).alias("max-min_" + str_window + "_LP_P3-O1"),



        # RP: Fp2 - F4
        pl.col("RP_Fp2-F4").mean().alias("mean_" + str_window + "_RP_Fp2-F4"),
        pl.col("RP_Fp2-F4").max().alias("min_" + str_window + "_RP_Fp2-F4"),
        pl.col("RP_Fp2-F4").min().alias("max_" + str_window + "_RP_Fp2-F4"),
        pl.col("RP_Fp2-F4").std().alias("std_" + str_window + "_RP_Fp2-F4"),
        (pl.col("RP_Fp2-F4").max() - pl.col("RP_Fp2-F4").min()).alias("max-min_" + str_window + "_RP_Fp2-F4"),

        # RP: F4 - C4
        pl.col("RP_F4-C4").mean().alias("mean_" + str_window + "_RP_F4-C4"),
        pl.col("RP_F4-C4").max().alias("min_" + str_window + "_RP_F4-C4"),
        pl.col("RP_F4-C4").min().alias("max_" + str_window + "_RP_F4-C4"),
        pl.col("RP_F4-C4").std().alias("std_" + str_window + "_RP_F4-C4"),
        (pl.col("RP_F4-C4").max() - pl.col("RP_F4-C4").min()).alias("max-min_" + str_window + "_RP_F4-C4"),

        # RP: C4 - P4
        pl.col("RP_C4-P4").mean().alias("mean_" + str_window + "_RP_C4-P4"),
        pl.col("RP_C4-P4").max().alias("min_" + str_window + "_RP_C4-P4"),
        pl.col("RP_C4-P4").min().alias("max_" + str_window + "_RP_C4-P4"),
        pl.col("RP_C4-P4").std().alias("std_" + str_window + "_RP_C4-P4"),
        (pl.col("RP_C4-P4").max() - pl.col("RP_C4-P4").min()).alias("max-min_" + str_window + "_RP_C4-P4"),

        # RP: P4 - O2
        pl.col("RP_P4-O2").mean().alias("mean_" + str_window + "_RP_P4-O2"),
        pl.col("RP_P4-O2").max().alias("min_" + str_window + "_RP_P4-O2"),
        pl.col("RP_P4-O2").min().alias("max_" + str_window + "_RP_P4-O2"),
        pl.col("RP_P4-O2").std().alias("std_" + str_window + "_RP_P4-O2"),
        (pl.col("RP_P4-O2").max() - pl.col("RP_P4-O2").min()).alias("max-min_" + str_window + "_RP_P4-O2"),


        
        # RR: Fp2 - F8
        pl.col("RR_Fp2-F8").mean().alias("mean_" + str_window + "_RR_Fp2-F8"),
        pl.col("RR_Fp2-F8").max().alias("min_" + str_window + "_RR_Fp2-F8"),
        pl.col("RR_Fp2-F8").min().alias("max_" + str_window + "_RR_Fp2-F8"),
        pl.col("RR_Fp2-F8").std().alias("std_" + str_window + "_RR_Fp2-F8"),
        (pl.col("RR_Fp2-F8").max() - pl.col("RR_Fp2-F8").min()).alias("max-min_" + str_window + "_RR_Fp2-F8"),

        # RR: F8 - T4
        pl.col("RR_F8-T4").mean().alias("mean_" + str_window + "_RR_F8-T4"),
        pl.col("RR_F8-T4").max().alias("min_" + str_window + "_RR_F8-T4"),
        pl.col("RR_F8-T4").min().alias("max_" + str_window + "_RR_F8-T4"),
        pl.col("RR_F8-T4").std().alias("std_" + str_window + "_RR_F8-T4"),
        (pl.col("RR_F8-T4").max() - pl.col("RR_F8-T4").min()).alias("max-min_" + str_window + "_RR_F8-T4"),

        # RR: T4 - T6
        pl.col("RR_T4-T6").mean().alias("mean_" + str_window + "_RR_T4-T6"),
        pl.col("RR_T4-T6").max().alias("min_" + str_window + "_RR_T4-T6"),
        pl.col("RR_T4-T6").min().alias("max_" + str_window + "_RR_T4-T6"),
        pl.col("RR_T4-T6").std().alias("std_" + str_window + "_RR_T4-T6"),
        (pl.col("RR_T4-T6").max() - pl.col("RR_T4-T6").min()).alias("max-min_" + str_window + "_RR_T4-T6"),

        # RR: T6 - O2
        pl.col("RR_T6-O2").mean().alias("mean_" + str_window + "_RR_T6-O2"),
        pl.col("RR_T6-O2").max().alias("min_" + str_window + "_RR_T6-O2"),
        pl.col("RR_T6-O2").min().alias("max_" + str_window + "_RR_T6-O2"),
        pl.col("RR_T6-O2").std().alias("std_" + str_window + "_RR_T6-O2"),
        (pl.col("RR_T6-O2").max() - pl.col("RR_T6-O2").min()).alias("max-min_" + str_window + "_RR_T6-O2"),        
    )

    
    # 差分特徴量をdrop
    df_features = df_features.drop(
        ['LL_Fp1-T3', 'LL_T3-O1', 'LP_Fp1-C3', 'LP_C3-O1', 'RP_Fp2-C4', 'RP_C4-O2', 'RR_Fp2-T4', 'RR_T4-O2',
         'LL_Fp1-F7', 'LL_F7-T3', 'LL_T3-T5', 'LL_T5-O1',
         'LP_Fp1-F3', 'LP_F3-C3', 'LP_C3-P3', 'LP_P3-O1',
         'RP_Fp2-F4', 'RP_F4-C4', 'RP_C4-P4', 'RP_P4-O2',
         'RR_Fp2-F8', 'RR_F8-T4', 'RR_T4-T6', 'RR_T6-O2',]
        )
    
    
    # 列名を一括で変更
    col_suffix = "_at_" + str(center_sec) + "s"
    list_original_col = df_features.columns
    df_features = df_features.with_columns(pl.all().name.suffix(col_suffix)).drop(list_original_col) # suffixを付与して、元の列名の列を削除
    
    # eeg_id 列の列名を戻す
    eeg_col = "eeg_id_at_" + str(center_sec) + "s"
    df_features = df_features.rename({eeg_col : "eeg_id"})


    return df_features

In [57]:
%%time
PATH = '../input/hms-harmful-brain-activity-classification/train_eegs/'
files = os.listdir(PATH)
print(f'There are {len(files)} eeg parquet files')

start = time.time()
df_all_diff_features = pl.DataFrame()
for i, f in enumerate(files):
    if i%100 == 0:
        print(i, ', ', end = '')
        
        ### 状況確認
        # 時間計測
        end = time.time()
        seconds = end - start
        seconds = int(seconds + 0.5)    # 秒数を四捨五入
        h = seconds // 3600             # 時の取得
        m = (seconds - h * 3600) // 60  # 分の取得
        s = seconds - h * 3600 - m * 60 # 秒の取得
        print(f"{h:02}:{m:02}:{s:02}" )
        start = time.time()

        # 形状確認
        print(df_all_diff_features.shape)

    # if i%10 != 0: continue

    eeg_id = int(f.split('.')[0])    

    # 読み込み
    temp_eeg = pl.read_parquet(f'{PATH}{f}')

    # eeg_id を追加
    temp_eeg = temp_eeg.with_columns(pl.lit(str(eeg_id)).alias("eeg_id"))

    # eegデータにおける、計測開始から各行までの秒数 seconds_eeg
    t = 1/200 # EEGのサンプリング周波数は200Hzなので、1行は1/200秒ごとの計測値を表す
    temp_eeg = temp_eeg.with_row_count("id")
    temp_eeg = temp_eeg.with_columns((pl.col("id") * t).alias("seconds_eeg"))   # 計測時刻の列をつくる
    temp_eeg = temp_eeg.filter(pl.col("seconds_eeg") < 50).drop(["id"])         # 50秒以降のデータは使わない
    
    # 差分特徴量を作る
    temp_eeg = temp_eeg.with_columns(
        # 簡易特徴量
        (pl.col("Fp1") - pl.col("T3")).alias("LL_Fp1-T3"),
        (pl.col("T3") - pl.col("O1")).alias("LL_T3-O1"),
        (pl.col("Fp1") - pl.col("C3")).alias("LP_Fp1-C3"),
        (pl.col("C3") - pl.col("O1")).alias("LP_C3-O1"),
        (pl.col("Fp2") - pl.col("C4")).alias("RP_Fp2-C4"),
        (pl.col("C4") - pl.col("O2")).alias("RP_C4-O2"),
        (pl.col("Fp2") - pl.col("T4")).alias("RR_Fp2-T4"),
        (pl.col("T4") - pl.col("O2")).alias("RR_T4-O2"),

        # 全差分の特徴量
        (pl.col("Fp1") - pl.col("F7")).alias("LL_Fp1-F7"),
        (pl.col("F7") - pl.col("T3")).alias("LL_F7-T3"),
        (pl.col("T3") - pl.col("T5")).alias("LL_T3-T5"),
        (pl.col("T5") - pl.col("O1")).alias("LL_T5-O1"),

        (pl.col("Fp1") - pl.col("F3")).alias("LP_Fp1-F3"),
        (pl.col("F3") - pl.col("C3")).alias("LP_F3-C3"),
        (pl.col("C3") - pl.col("P3")).alias("LP_C3-P3"),
        (pl.col("P3") - pl.col("O1")).alias("LP_P3-O1"),

        (pl.col("Fp2") - pl.col("F4")).alias("RP_Fp2-F4"),
        (pl.col("F4") - pl.col("C4")).alias("RP_F4-C4"),
        (pl.col("C4") - pl.col("P4")).alias("RP_C4-P4"),
        (pl.col("P4") - pl.col("O2")).alias("RP_P4-O2"),

        (pl.col("Fp2") - pl.col("F8")).alias("RR_Fp2-F8"),
        (pl.col("F8") - pl.col("T4")).alias("RR_F8-T4"),
        (pl.col("T4") - pl.col("T6")).alias("RR_T4-T6"),
        (pl.col("T6") - pl.col("O2")).alias("RR_T6-O2"),
        
    )

    # 差分特徴量以外drop
    temp_eeg = temp_eeg.drop(['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2', 'EKG'])
    
    # eeg_id のみ持つデータフレームを作成
    df_eeg_feature = pl.DataFrame({"eeg_id": str(eeg_id)})

    # ひとつのeeg_id のデータフレームに対して特徴量を作成
    for center_sec, window in zip(center_seconds_list, window_list):
            df_features = create_aggregated_features_on_EEG_differences(temp_eeg, window, center_sec)
            df_eeg_feature = df_eeg_feature.join(df_features, on = "eeg_id", how = "left")

    # 全ての eeg_id に対して結合
    df_all_diff_features = pl.concat([df_all_diff_features, df_eeg_feature], how = 'vertical')

There are 17300 eeg parquet files
0 , 00:00:00
(0, 0)
100 , 00:00:00
(10, 1201)
200 , 00:00:00
(20, 1201)
300 , 00:00:00
(30, 1201)
400 , 00:00:00
(40, 1201)
500 , 00:00:00
(50, 1201)
600 , 00:00:00
(60, 1201)
700 , 00:00:00
(70, 1201)
800 , 00:00:00
(80, 1201)
900 , 00:00:00
(90, 1201)
1000 , 00:00:00
(100, 1201)
1100 , 00:00:00
(110, 1201)
1200 , 00:00:00
(120, 1201)
1300 , 00:00:00
(130, 1201)
1400 , 00:00:00
(140, 1201)
1500 , 00:00:00
(150, 1201)
1600 , 00:00:00
(160, 1201)
1700 , 00:00:00
(170, 1201)
1800 , 00:00:00
(180, 1201)
1900 , 00:00:00
(190, 1201)
2000 , 00:00:00
(200, 1201)
2100 , 00:00:00
(210, 1201)
2200 , 00:00:00
(220, 1201)
2300 , 00:00:00
(230, 1201)
2400 , 00:00:00
(240, 1201)
2500 , 00:00:00
(250, 1201)
2600 , 00:00:00
(260, 1201)
2700 , 00:00:00
(270, 1201)
2800 , 00:00:00
(280, 1201)
2900 , 00:00:00
(290, 1201)
3000 , 00:00:00
(300, 1201)
3100 , 00:00:00
(310, 1201)
3200 , 00:00:00
(320, 1201)
3300 , 00:00:00
(330, 1201)
3400 , 00:00:00
(340, 1201)
3500 , 00:00

In [58]:
df_all_diff_features.head()

eeg_id,mean_5s_LL_Fp1-T3_at_10s,min_5s_LL_Fp1-T3_at_10s,max_5s_LL_Fp1-T3_at_10s,std_5s_LL_Fp1-T3_at_10s,max-min_5s_LL_Fp1-T3_at_10s,mean_5s_LL_T3-O1_at_10s,min_5s_LL_T3-O1_at_10s,max_5s_LL_T3-O1_at_10s,std_5s_LL_T3-O1_at_10s,max-min__5s_LL_T3-O1_at_10s,mean_5s_LP_Fp1-C3_at_10s,min_5s_LP_Fp1-C3_at_10s,max_5s_LP_Fp1-C3_at_10s,std_5s_LP_Fp1-C3_at_10s,max-min_5s_LP_Fp1-C3_at_10s,mean_5s_LP_C3-O1_at_10s,min_5s_LP_C3-O1_at_10s,max_5s_LP_C3-O1_at_10s,std_5s_LP_C3-O1_at_10s,max-min_5s_LP_C3-O1_at_10s,mean_5s_RP_Fp2-C4_at_10s,min_5s_RP_Fp2-C4_at_10s,max_5s_RP_Fp2-C4_at_10s,std_5s_RP_Fp2-C4_at_10s,max-min_5s_RP_Fp2-C4_at_10s,mean_5s_RP_C4-O2_at_10s,min_5s_RP_C4-O2_at_10s,max_5s_RP_C4-O2_at_10s,std_5s_RP_C4-O2_at_10s,max-min_5s_RP_C4-O2_at_10s,mean_5s_RR_Fp2-T4_at_10s,min_5s_RR_Fp2-T4_at_10s,max_5s_RR_Fp2-T4_at_10s,std_5s_RR_Fp2-T4_at_10s,max-min_5s_RR_Fp2-T4_at_10s,mean_5s_RR_T4-O2_at_10s,min_5s_RR_T4-O2_at_10s,max_5s_RR_T4-O2_at_10s,std_5s_RR_T4-O2_at_10s,max-min_5s_RR_T4-O2_at_10s,mean_5s_LL_Fp1-F7_at_10s,min_5s_LL_Fp1-F7_at_10s,max_5s_LL_Fp1-F7_at_10s,std_5s_LL_Fp1-F7_at_10s,max-min_5s_LL_Fp1-F7_at_10s,mean_5s_LL_F7-T3_at_10s,min_5s_LL_F7-T3_at_10s,max_5s_LL_F7-T3_at_10s,std_5s_LL_F7-T3_at_10s,max-min_5s_LL_F7-T3_at_10s,mean_5s_LL_T3-T5_at_10s,min_5s_LL_T3-T5_at_10s,max_5s_LL_T3-T5_at_10s,std_5s_LL_T3-T5_at_10s,max-min_5s_LL_T3-T5_at_10s,mean_5s_LL_T5-O1_at_10s,min_5s_LL_T5-O1_at_10s,max_5s_LL_T5-O1_at_10s,std_5s_LL_T5-O1_at_10s,max-min_5s_LL_T5-O1_at_10s,mean_5s_LP_Fp1-F3_at_10s,min_5s_LP_Fp1-F3_at_10s,max_5s_LP_Fp1-F3_at_10s,std_5s_LP_Fp1-F3_at_10s,max-min_5s_LP_Fp1-F3_at_10s,mean_5s_LP_F3-C3_at_10s,min_5s_LP_F3-C3_at_10s,max_5s_LP_F3-C3_at_10s,std_5s_LP_F3-C3_at_10s,max-min_5s_LP_F3-C3_at_10s,mean_5s_LP_C3-P3_at_10s,min_5s_LP_C3-P3_at_10s,max_5s_LP_C3-P3_at_10s,std_5s_LP_C3-P3_at_10s,max-min_5s_LP_C3-P3_at_10s,mean_5s_LP_P3-O1_at_10s,min_5s_LP_P3-O1_at_10s,max_5s_LP_P3-O1_at_10s,std_5s_LP_P3-O1_at_10s,max-min_5s_LP_P3-O1_at_10s,mean_5s_RP_Fp2-F4_at_10s,min_5s_RP_Fp2-F4_at_10s,max_5s_RP_Fp2-F4_at_10s,std_5s_RP_Fp2-F4_at_10s,max-min_5s_RP_Fp2-F4_at_10s,mean_5s_RP_F4-C4_at_10s,min_5s_RP_F4-C4_at_10s,max_5s_RP_F4-C4_at_10s,std_5s_RP_F4-C4_at_10s,max-min_5s_RP_F4-C4_at_10s,mean_5s_RP_C4-P4_at_10s,min_5s_RP_C4-P4_at_10s,max_5s_RP_C4-P4_at_10s,std_5s_RP_C4-P4_at_10s,max-min_5s_RP_C4-P4_at_10s,mean_5s_RP_P4-O2_at_10s,min_5s_RP_P4-O2_at_10s,max_5s_RP_P4-O2_at_10s,std_5s_RP_P4-O2_at_10s,max-min_5s_RP_P4-O2_at_10s,mean_5s_RR_Fp2-F8_at_10s,min_5s_RR_Fp2-F8_at_10s,max_5s_RR_Fp2-F8_at_10s,std_5s_RR_Fp2-F8_at_10s,max-min_5s_RR_Fp2-F8_at_10s,mean_5s_RR_F8-T4_at_10s,min_5s_RR_F8-T4_at_10s,max_5s_RR_F8-T4_at_10s,std_5s_RR_F8-T4_at_10s,max-min_5s_RR_F8-T4_at_10s,mean_5s_RR_T4-T6_at_10s,min_5s_RR_T4-T6_at_10s,max_5s_RR_T4-T6_at_10s,std_5s_RR_T4-T6_at_10s,max-min_5s_RR_T4-T6_at_10s,mean_5s_RR_T6-O2_at_10s,min_5s_RR_T6-O2_at_10s,max_5s_RR_T6-O2_at_10s,std_5s_RR_T6-O2_at_10s,max-min_5s_RR_T6-O2_at_10s,mean_10s_LL_Fp1-T3_at_10s,min_10s_LL_Fp1-T3_at_10s,max_10s_LL_Fp1-T3_at_10s,std_10s_LL_Fp1-T3_at_10s,max-min_10s_LL_Fp1-T3_at_10s,mean_10s_LL_T3-O1_at_10s,min_10s_LL_T3-O1_at_10s,max_10s_LL_T3-O1_at_10s,std_10s_LL_T3-O1_at_10s,max-min__10s_LL_T3-O1_at_10s,mean_10s_LP_Fp1-C3_at_10s,min_10s_LP_Fp1-C3_at_10s,max_10s_LP_Fp1-C3_at_10s,std_10s_LP_Fp1-C3_at_10s,max-min_10s_LP_Fp1-C3_at_10s,mean_10s_LP_C3-O1_at_10s,min_10s_LP_C3-O1_at_10s,max_10s_LP_C3-O1_at_10s,std_10s_LP_C3-O1_at_10s,max-min_10s_LP_C3-O1_at_10s,mean_10s_RP_Fp2-C4_at_10s,min_10s_RP_Fp2-C4_at_10s,max_10s_RP_Fp2-C4_at_10s,std_10s_RP_Fp2-C4_at_10s,max-min_10s_RP_Fp2-C4_at_10s,mean_10s_RP_C4-O2_at_10s,min_10s_RP_C4-O2_at_10s,max_10s_RP_C4-O2_at_10s,std_10s_RP_C4-O2_at_10s,max-min_10s_RP_C4-O2_at_10s,mean_10s_RR_Fp2-T4_at_10s,min_10s_RR_Fp2-T4_at_10s,max_10s_RR_Fp2-T4_at_10s,std_10s_RR_Fp2-T4_at_10s,max-min_10s_RR_Fp2-T4_at_10s,mean_10s_RR_T4-O2_at_10s,min_10s_RR_T4-O2_at_10s,max_10s_RR_T4-O2_at_10s,std_10s_RR_T4-O2_at_10s,max-min_10s_RR_T4-O2_at_10s,mean_10s_LL_Fp1-F7_at_10s,min_10s_LL_Fp1-F7_at_10s,

In [59]:
df_all_diff_features = df_all_diff_features.with_columns(pl.col("eeg_id").cast(pl.Int64).alias("eeg_id"))
df_all_diff_features.tail(5)

eeg_id,mean_5s_LL_Fp1-T3_at_10s,min_5s_LL_Fp1-T3_at_10s,max_5s_LL_Fp1-T3_at_10s,std_5s_LL_Fp1-T3_at_10s,max-min_5s_LL_Fp1-T3_at_10s,mean_5s_LL_T3-O1_at_10s,min_5s_LL_T3-O1_at_10s,max_5s_LL_T3-O1_at_10s,std_5s_LL_T3-O1_at_10s,max-min__5s_LL_T3-O1_at_10s,mean_5s_LP_Fp1-C3_at_10s,min_5s_LP_Fp1-C3_at_10s,max_5s_LP_Fp1-C3_at_10s,std_5s_LP_Fp1-C3_at_10s,max-min_5s_LP_Fp1-C3_at_10s,mean_5s_LP_C3-O1_at_10s,min_5s_LP_C3-O1_at_10s,max_5s_LP_C3-O1_at_10s,std_5s_LP_C3-O1_at_10s,max-min_5s_LP_C3-O1_at_10s,mean_5s_RP_Fp2-C4_at_10s,min_5s_RP_Fp2-C4_at_10s,max_5s_RP_Fp2-C4_at_10s,std_5s_RP_Fp2-C4_at_10s,max-min_5s_RP_Fp2-C4_at_10s,mean_5s_RP_C4-O2_at_10s,min_5s_RP_C4-O2_at_10s,max_5s_RP_C4-O2_at_10s,std_5s_RP_C4-O2_at_10s,max-min_5s_RP_C4-O2_at_10s,mean_5s_RR_Fp2-T4_at_10s,min_5s_RR_Fp2-T4_at_10s,max_5s_RR_Fp2-T4_at_10s,std_5s_RR_Fp2-T4_at_10s,max-min_5s_RR_Fp2-T4_at_10s,mean_5s_RR_T4-O2_at_10s,min_5s_RR_T4-O2_at_10s,max_5s_RR_T4-O2_at_10s,std_5s_RR_T4-O2_at_10s,max-min_5s_RR_T4-O2_at_10s,mean_5s_LL_Fp1-F7_at_10s,min_5s_LL_Fp1-F7_at_10s,max_5s_LL_Fp1-F7_at_10s,std_5s_LL_Fp1-F7_at_10s,max-min_5s_LL_Fp1-F7_at_10s,mean_5s_LL_F7-T3_at_10s,min_5s_LL_F7-T3_at_10s,max_5s_LL_F7-T3_at_10s,std_5s_LL_F7-T3_at_10s,max-min_5s_LL_F7-T3_at_10s,mean_5s_LL_T3-T5_at_10s,min_5s_LL_T3-T5_at_10s,max_5s_LL_T3-T5_at_10s,std_5s_LL_T3-T5_at_10s,max-min_5s_LL_T3-T5_at_10s,mean_5s_LL_T5-O1_at_10s,min_5s_LL_T5-O1_at_10s,max_5s_LL_T5-O1_at_10s,std_5s_LL_T5-O1_at_10s,max-min_5s_LL_T5-O1_at_10s,mean_5s_LP_Fp1-F3_at_10s,min_5s_LP_Fp1-F3_at_10s,max_5s_LP_Fp1-F3_at_10s,std_5s_LP_Fp1-F3_at_10s,max-min_5s_LP_Fp1-F3_at_10s,mean_5s_LP_F3-C3_at_10s,min_5s_LP_F3-C3_at_10s,max_5s_LP_F3-C3_at_10s,std_5s_LP_F3-C3_at_10s,max-min_5s_LP_F3-C3_at_10s,mean_5s_LP_C3-P3_at_10s,min_5s_LP_C3-P3_at_10s,max_5s_LP_C3-P3_at_10s,std_5s_LP_C3-P3_at_10s,max-min_5s_LP_C3-P3_at_10s,mean_5s_LP_P3-O1_at_10s,min_5s_LP_P3-O1_at_10s,max_5s_LP_P3-O1_at_10s,std_5s_LP_P3-O1_at_10s,max-min_5s_LP_P3-O1_at_10s,mean_5s_RP_Fp2-F4_at_10s,min_5s_RP_Fp2-F4_at_10s,max_5s_RP_Fp2-F4_at_10s,std_5s_RP_Fp2-F4_at_10s,max-min_5s_RP_Fp2-F4_at_10s,mean_5s_RP_F4-C4_at_10s,min_5s_RP_F4-C4_at_10s,max_5s_RP_F4-C4_at_10s,std_5s_RP_F4-C4_at_10s,max-min_5s_RP_F4-C4_at_10s,mean_5s_RP_C4-P4_at_10s,min_5s_RP_C4-P4_at_10s,max_5s_RP_C4-P4_at_10s,std_5s_RP_C4-P4_at_10s,max-min_5s_RP_C4-P4_at_10s,mean_5s_RP_P4-O2_at_10s,min_5s_RP_P4-O2_at_10s,max_5s_RP_P4-O2_at_10s,std_5s_RP_P4-O2_at_10s,max-min_5s_RP_P4-O2_at_10s,mean_5s_RR_Fp2-F8_at_10s,min_5s_RR_Fp2-F8_at_10s,max_5s_RR_Fp2-F8_at_10s,std_5s_RR_Fp2-F8_at_10s,max-min_5s_RR_Fp2-F8_at_10s,mean_5s_RR_F8-T4_at_10s,min_5s_RR_F8-T4_at_10s,max_5s_RR_F8-T4_at_10s,std_5s_RR_F8-T4_at_10s,max-min_5s_RR_F8-T4_at_10s,mean_5s_RR_T4-T6_at_10s,min_5s_RR_T4-T6_at_10s,max_5s_RR_T4-T6_at_10s,std_5s_RR_T4-T6_at_10s,max-min_5s_RR_T4-T6_at_10s,mean_5s_RR_T6-O2_at_10s,min_5s_RR_T6-O2_at_10s,max_5s_RR_T6-O2_at_10s,std_5s_RR_T6-O2_at_10s,max-min_5s_RR_T6-O2_at_10s,mean_10s_LL_Fp1-T3_at_10s,min_10s_LL_Fp1-T3_at_10s,max_10s_LL_Fp1-T3_at_10s,std_10s_LL_Fp1-T3_at_10s,max-min_10s_LL_Fp1-T3_at_10s,mean_10s_LL_T3-O1_at_10s,min_10s_LL_T3-O1_at_10s,max_10s_LL_T3-O1_at_10s,std_10s_LL_T3-O1_at_10s,max-min__10s_LL_T3-O1_at_10s,mean_10s_LP_Fp1-C3_at_10s,min_10s_LP_Fp1-C3_at_10s,max_10s_LP_Fp1-C3_at_10s,std_10s_LP_Fp1-C3_at_10s,max-min_10s_LP_Fp1-C3_at_10s,mean_10s_LP_C3-O1_at_10s,min_10s_LP_C3-O1_at_10s,max_10s_LP_C3-O1_at_10s,std_10s_LP_C3-O1_at_10s,max-min_10s_LP_C3-O1_at_10s,mean_10s_RP_Fp2-C4_at_10s,min_10s_RP_Fp2-C4_at_10s,max_10s_RP_Fp2-C4_at_10s,std_10s_RP_Fp2-C4_at_10s,max-min_10s_RP_Fp2-C4_at_10s,mean_10s_RP_C4-O2_at_10s,min_10s_RP_C4-O2_at_10s,max_10s_RP_C4-O2_at_10s,std_10s_RP_C4-O2_at_10s,max-min_10s_RP_C4-O2_at_10s,mean_10s_RR_Fp2-T4_at_10s,min_10s_RR_Fp2-T4_at_10s,max_10s_RR_Fp2-T4_at_10s,std_10s_RR_Fp2-T4_at_10s,max-min_10s_RR_Fp2-T4_at_10s,mean_10s_RR_T4-O2_at_10s,min_10s_RR_T4-O2_at_10s,max_10s_RR_T4-O2_at_10s,std_10s_RR_T4-O2_at_10s,max-min_10s_RR_T4-O2_at_10s,mean_10s_LL_Fp1-F7_at_10s,min_10s_LL_Fp1-F7_at_10s,

## 3. 各特徴量をjoin

In [60]:
# 集計特徴量をjoinc
df_train = df_train.join(df_all_diff_features, on = "eeg_id", how = "left")
df_train.head()

eeg_id,spec_id,eeg_min_offset,eeg_max_offset,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target,mean_5s_LL_Fp1-T3_at_10s,min_5s_LL_Fp1-T3_at_10s,max_5s_LL_Fp1-T3_at_10s,std_5s_LL_Fp1-T3_at_10s,max-min_5s_LL_Fp1-T3_at_10s,mean_5s_LL_T3-O1_at_10s,min_5s_LL_T3-O1_at_10s,max_5s_LL_T3-O1_at_10s,std_5s_LL_T3-O1_at_10s,max-min__5s_LL_T3-O1_at_10s,mean_5s_LP_Fp1-C3_at_10s,min_5s_LP_Fp1-C3_at_10s,max_5s_LP_Fp1-C3_at_10s,std_5s_LP_Fp1-C3_at_10s,max-min_5s_LP_Fp1-C3_at_10s,mean_5s_LP_C3-O1_at_10s,min_5s_LP_C3-O1_at_10s,max_5s_LP_C3-O1_at_10s,std_5s_LP_C3-O1_at_10s,max-min_5s_LP_C3-O1_at_10s,mean_5s_RP_Fp2-C4_at_10s,min_5s_RP_Fp2-C4_at_10s,max_5s_RP_Fp2-C4_at_10s,std_5s_RP_Fp2-C4_at_10s,max-min_5s_RP_Fp2-C4_at_10s,mean_5s_RP_C4-O2_at_10s,min_5s_RP_C4-O2_at_10s,max_5s_RP_C4-O2_at_10s,std_5s_RP_C4-O2_at_10s,max-min_5s_RP_C4-O2_at_10s,mean_5s_RR_Fp2-T4_at_10s,min_5s_RR_Fp2-T4_at_10s,max_5s_RR_Fp2-T4_at_10s,std_5s_RR_Fp2-T4_at_10s,max-min_5s_RR_Fp2-T4_at_10s,mean_5s_RR_T4-O2_at_10s,min_5s_RR_T4-O2_at_10s,max_5s_RR_T4-O2_at_10s,std_5s_RR_T4-O2_at_10s,max-min_5s_RR_T4-O2_at_10s,mean_5s_LL_Fp1-F7_at_10s,min_5s_LL_Fp1-F7_at_10s,max_5s_LL_Fp1-F7_at_10s,std_5s_LL_Fp1-F7_at_10s,max-min_5s_LL_Fp1-F7_at_10s,mean_5s_LL_F7-T3_at_10s,min_5s_LL_F7-T3_at_10s,max_5s_LL_F7-T3_at_10s,std_5s_LL_F7-T3_at_10s,max-min_5s_LL_F7-T3_at_10s,mean_5s_LL_T3-T5_at_10s,min_5s_LL_T3-T5_at_10s,max_5s_LL_T3-T5_at_10s,std_5s_LL_T3-T5_at_10s,max-min_5s_LL_T3-T5_at_10s,mean_5s_LL_T5-O1_at_10s,min_5s_LL_T5-O1_at_10s,max_5s_LL_T5-O1_at_10s,std_5s_LL_T5-O1_at_10s,max-min_5s_LL_T5-O1_at_10s,mean_5s_LP_Fp1-F3_at_10s,min_5s_LP_Fp1-F3_at_10s,max_5s_LP_Fp1-F3_at_10s,std_5s_LP_Fp1-F3_at_10s,max-min_5s_LP_Fp1-F3_at_10s,mean_5s_LP_F3-C3_at_10s,min_5s_LP_F3-C3_at_10s,max_5s_LP_F3-C3_at_10s,std_5s_LP_F3-C3_at_10s,max-min_5s_LP_F3-C3_at_10s,mean_5s_LP_C3-P3_at_10s,min_5s_LP_C3-P3_at_10s,max_5s_LP_C3-P3_at_10s,std_5s_LP_C3-P3_at_10s,max-min_5s_LP_C3-P3_at_10s,mean_5s_LP_P3-O1_at_10s,min_5s_LP_P3-O1_at_10s,max_5s_LP_P3-O1_at_10s,std_5s_LP_P3-O1_at_10s,max-min_5s_LP_P3-O1_at_10s,mean_5s_RP_Fp2-F4_at_10s,min_5s_RP_Fp2-F4_at_10s,max_5s_RP_Fp2-F4_at_10s,std_5s_RP_Fp2-F4_at_10s,max-min_5s_RP_Fp2-F4_at_10s,mean_5s_RP_F4-C4_at_10s,min_5s_RP_F4-C4_at_10s,max_5s_RP_F4-C4_at_10s,std_5s_RP_F4-C4_at_10s,max-min_5s_RP_F4-C4_at_10s,mean_5s_RP_C4-P4_at_10s,min_5s_RP_C4-P4_at_10s,max_5s_RP_C4-P4_at_10s,std_5s_RP_C4-P4_at_10s,max-min_5s_RP_C4-P4_at_10s,mean_5s_RP_P4-O2_at_10s,min_5s_RP_P4-O2_at_10s,max_5s_RP_P4-O2_at_10s,std_5s_RP_P4-O2_at_10s,max-min_5s_RP_P4-O2_at_10s,mean_5s_RR_Fp2-F8_at_10s,min_5s_RR_Fp2-F8_at_10s,max_5s_RR_Fp2-F8_at_10s,std_5s_RR_Fp2-F8_at_10s,max-min_5s_RR_Fp2-F8_at_10s,mean_5s_RR_F8-T4_at_10s,min_5s_RR_F8-T4_at_10s,max_5s_RR_F8-T4_at_10s,std_5s_RR_F8-T4_at_10s,max-min_5s_RR_F8-T4_at_10s,mean_5s_RR_T4-T6_at_10s,min_5s_RR_T4-T6_at_10s,max_5s_RR_T4-T6_at_10s,std_5s_RR_T4-T6_at_10s,max-min_5s_RR_T4-T6_at_10s,mean_5s_RR_T6-O2_at_10s,min_5s_RR_T6-O2_at_10s,max_5s_RR_T6-O2_at_10s,std_5s_RR_T6-O2_at_10s,max-min_5s_RR_T6-O2_at_10s,mean_10s_LL_Fp1-T3_at_10s,min_10s_LL_Fp1-T3_at_10s,max_10s_LL_Fp1-T3_at_10s,std_10s_LL_Fp1-T3_at_10s,max-min_10s_LL_Fp1-T3_at_10s,mean_10s_LL_T3-O1_at_10s,min_10s_LL_T3-O1_at_10s,max_10s_LL_T3-O1_at_10s,std_10s_LL_T3-O1_at_10s,max-min__10s_LL_T3-O1_at_10s,mean_10s_LP_Fp1-C3_at_10s,min_10s_LP_Fp1-C3_at_10s,max_10s_LP_Fp1-C3_at_10s,std_10s_LP_Fp1-C3_at_10s,max-min_10s_LP_Fp1-C3_at_10s,mean_10s_LP_C3-O1_at_10s,min_10s_LP_C3-O1_at_10s,max_10s_LP_C3-O1_at_10s,std_10s_LP_C3-O1_at_10s,max-min_10s_LP_C3-O1_at_10s,mean_10s_RP_Fp2-C4_at_10s,min_10s_RP_Fp2-C4_at_10s,max_10s_RP_Fp2-C4_at_10s,std_10s_RP_Fp2-C4_at_10s,max-min_10s_RP_Fp2-C4_at_10s,mean_10s_RP_C4-O2_at_10s,min_10s_RP_C4-O2_at_10s,max_10s_RP_C4-O2_at_10s,std_10s_RP_C4-O2_at_10s,max-min_10s_RP_C4-O2_at_10s,mean_10s_RR_Fp2-T4_at_10s,min_10s_RR_Fp2-T4_at_10s,max_10s_RR_Fp2-T4_at_10s,std_10s_RR_Fp2-T4_at_10s,max-min_10s_RR_Fp2-T4_at_10s,mean_10s_RR_T4-O2_at_10s,min_10s_RR_T4-O2_at_10s,max_10s_R

In [61]:
# spec特徴量をjoin
train_spec = pl.from_pandas(train_spec)
df_train = df_train.join(train_spec, on = "eeg_id", how = "left")

## 4. CatBoostに入れるための準備

In [62]:
# Testデータでは、eeg_id, patient_id, spectrogram_id, label_idは1つしかないため、trainingにも使わない
# patient_idについて、データの分割に利用
FEATURES = df_train.drop(["eeg_id", "eeg_sub_id", "eeg_label_offset_seconds", "eeg_max_offset", "eeg_min_offset",
                           "spec_id", "spectrogram_id", "spectrogram_sub_id", "spectrogram_label_offset_seconds",
                           "patient_id", "label_id", "seconds_eeg",
                           "seizure_vote", "lpd_vote", "gpd_vote", "lrda_vote", "grda_vote", "other_vote", "vote_sum",
                           "EKG", "target",
                           ]).columns
FEATURES

['mean_5s_LL_Fp1-T3_at_10s',
 'min_5s_LL_Fp1-T3_at_10s',
 'max_5s_LL_Fp1-T3_at_10s',
 'std_5s_LL_Fp1-T3_at_10s',
 'max-min_5s_LL_Fp1-T3_at_10s',
 'mean_5s_LL_T3-O1_at_10s',
 'min_5s_LL_T3-O1_at_10s',
 'max_5s_LL_T3-O1_at_10s',
 'std_5s_LL_T3-O1_at_10s',
 'max-min__5s_LL_T3-O1_at_10s',
 'mean_5s_LP_Fp1-C3_at_10s',
 'min_5s_LP_Fp1-C3_at_10s',
 'max_5s_LP_Fp1-C3_at_10s',
 'std_5s_LP_Fp1-C3_at_10s',
 'max-min_5s_LP_Fp1-C3_at_10s',
 'mean_5s_LP_C3-O1_at_10s',
 'min_5s_LP_C3-O1_at_10s',
 'max_5s_LP_C3-O1_at_10s',
 'std_5s_LP_C3-O1_at_10s',
 'max-min_5s_LP_C3-O1_at_10s',
 'mean_5s_RP_Fp2-C4_at_10s',
 'min_5s_RP_Fp2-C4_at_10s',
 'max_5s_RP_Fp2-C4_at_10s',
 'std_5s_RP_Fp2-C4_at_10s',
 'max-min_5s_RP_Fp2-C4_at_10s',
 'mean_5s_RP_C4-O2_at_10s',
 'min_5s_RP_C4-O2_at_10s',
 'max_5s_RP_C4-O2_at_10s',
 'std_5s_RP_C4-O2_at_10s',
 'max-min_5s_RP_C4-O2_at_10s',
 'mean_5s_RR_Fp2-T4_at_10s',
 'min_5s_RR_Fp2-T4_at_10s',
 'max_5s_RR_Fp2-T4_at_10s',
 'std_5s_RR_Fp2-T4_at_10s',
 'max-min_5s_RR_Fp2-T4_at_10s',

In [63]:
df_train_pl = df_train

In [64]:
# pandasに変換
df_train = df_train_pl.to_pandas()

In [65]:
# target列以外のすべての列をfloatに変換
columns_to_convert = [col for col in df_train.columns if col != 'target']
df_train[columns_to_convert] = df_train[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [66]:
# Target変数のラベルの定義
TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
df_train['target'] = df_train['target'].replace(TARS)

In [67]:
FEATURES

['mean_5s_LL_Fp1-T3_at_10s',
 'min_5s_LL_Fp1-T3_at_10s',
 'max_5s_LL_Fp1-T3_at_10s',
 'std_5s_LL_Fp1-T3_at_10s',
 'max-min_5s_LL_Fp1-T3_at_10s',
 'mean_5s_LL_T3-O1_at_10s',
 'min_5s_LL_T3-O1_at_10s',
 'max_5s_LL_T3-O1_at_10s',
 'std_5s_LL_T3-O1_at_10s',
 'max-min__5s_LL_T3-O1_at_10s',
 'mean_5s_LP_Fp1-C3_at_10s',
 'min_5s_LP_Fp1-C3_at_10s',
 'max_5s_LP_Fp1-C3_at_10s',
 'std_5s_LP_Fp1-C3_at_10s',
 'max-min_5s_LP_Fp1-C3_at_10s',
 'mean_5s_LP_C3-O1_at_10s',
 'min_5s_LP_C3-O1_at_10s',
 'max_5s_LP_C3-O1_at_10s',
 'std_5s_LP_C3-O1_at_10s',
 'max-min_5s_LP_C3-O1_at_10s',
 'mean_5s_RP_Fp2-C4_at_10s',
 'min_5s_RP_Fp2-C4_at_10s',
 'max_5s_RP_Fp2-C4_at_10s',
 'std_5s_RP_Fp2-C4_at_10s',
 'max-min_5s_RP_Fp2-C4_at_10s',
 'mean_5s_RP_C4-O2_at_10s',
 'min_5s_RP_C4-O2_at_10s',
 'max_5s_RP_C4-O2_at_10s',
 'std_5s_RP_C4-O2_at_10s',
 'max-min_5s_RP_C4-O2_at_10s',
 'mean_5s_RR_Fp2-T4_at_10s',
 'min_5s_RR_Fp2-T4_at_10s',
 'max_5s_RR_Fp2-T4_at_10s',
 'std_5s_RR_Fp2-T4_at_10s',
 'max-min_5s_RR_Fp2-T4_at_10s',

In [68]:
df_train[FEATURES].shape

(17089, 4848)

In [69]:
list_temp = list(set(df_train.columns.to_list()) - set(FEATURES))
df_train[list_temp]

,lpd_vote,gpd_vote,spec_id,seizure_vote,other_vote,eeg_max_offset,grda_vote,eeg_min_offset,target,eeg_id,lrda_vote,patient_id
0,0.0,0.000000,1338193223,0.000000,1.000000,100.0,0.000000,0.0,5,1604920026,0.000000,60066
1,0.0,0.181818,2126560277,0.818182,0.000000,6.0,0.000000,0.0,0,990449207,0.000000,22597
2,0.0,0.000000,44306235,0.000000,1.000000,0.0,0.000000,0.0,5,3957673122,0.000000,9969
3,0.2,0.050000,107927759,0.000000,0.750000,2.0,0.000000,0.0,5,498482803,0.000000,13133
4,0.0,0.000000,174063666,1.000000,0.000000,10.0,0.000000,0.0,0,1235456176,0.000000,13732
...,...,...,...,...,...,...,...,...,...,...,...,...
17084,0.0,1.000000,936407961,0.000000,0.000000,0.0,0.000000,0.0,2,2967212512,0.000000,38549
17085,0.0,0.000000,561554038,0.000000,1.000000,6.0,0.000000,0.0,5,3907557307,0.000000,59059
17086,0.0,0.000000,306938429,0.000000,0.846154,2.0,0.153846,0.0,5,1086780865,0.000000,25986
17087,0.0,0.000000,841227593,0.000000,0.000000,98.0,0.000000,0.0,3,484920575,1.000000,62197


# Train CatBoost
CatBoost は、超高速トレーニングのために (パラメーター `task_type='GPU'` を追加すると) 両方の Kaggle T4 GPU を自動的に使用する。

In [70]:
import catboost as cat
from catboost import CatBoostClassifier, Pool
print('CatBoost version',cat.__version__)

CatBoost version 1.2.2


In [71]:
if not os.path.exists('models'):
    os.makedirs('models')

In [72]:
%%time
from sklearn.model_selection import KFold, GroupKFold
import json

all_oof = []
all_true = []

gkf = GroupKFold(n_splits=5)
for i, (train_index, valid_index) in enumerate(gkf.split(df_train, df_train.target, df_train.patient_id)):   
    
    print('#'*25)
    print(f'### Fold {i+1}')
    print(f'### train size {len(train_index)}, valid size {len(valid_index)}')
    print('#'*25)

    # model = CatBoostClassifier(loss_function = 'MultiClass')    
    model = CatBoostClassifier(task_type='GPU',
                               loss_function='MultiClass')
    
    # trainデータ / validデータのPoolを作成
    train_pool = Pool(
        data = df_train.loc[train_index,FEATURES],
        label = df_train.loc[train_index,'target'],
    )
    
    valid_pool = Pool(
        data = df_train.loc[valid_index,FEATURES],
        label = df_train.loc[valid_index,'target'],
    )
    
    model.fit(train_pool,
             verbose=100,
             eval_set=valid_pool,
             )
    
    # モデルの保存
    model.save_model(f'models/CAT_v{VER}_f{i}.cat')

    # モデルのパラメータの保存
    params = model.get_all_params()
    with open(f'models/CAT_v{VER}_f{i}_params.txt', 'w') as f:
        json.dump(params, f)

    # 各 fold の予測確率
    oof = model.predict_proba(valid_pool)
    all_oof.append(oof)
    all_true.append(df_train.loc[valid_index, TARGETS].values)
    
    del train_pool, valid_pool, oof #model
    gc.collect()
    
    #break
    
all_oof = np.concatenate(all_oof)   # 全validationデータに対する予測確率
all_true = np.concatenate(all_true) # 全validationデータのターゲット変数の実測値

#########################
### Fold 1
### train size 13671, valid size 3418
#########################
Learning rate set to 0.113774
0:	learn: 1.7338836	test: 1.7403037	best: 1.7403037 (0)	total: 1.67s	remaining: 27m 44s
100:	learn: 1.4633875	test: 1.5873333	best: 1.5861343 (89)	total: 2m 33s	remaining: 22m 42s
200:	learn: 1.4110876	test: 1.5873494	best: 1.5844244 (138)	total: 5m 8s	remaining: 20m 27s
300:	learn: 1.3783917	test: 1.5887148	best: 1.5844244 (138)	total: 7m 38s	remaining: 17m 45s
400:	learn: 1.3559176	test: 1.5911761	best: 1.5844244 (138)	total: 10m 10s	remaining: 15m 11s
500:	learn: 1.3391445	test: 1.5934460	best: 1.5844244 (138)	total: 12m 40s	remaining: 12m 37s
600:	learn: 1.3266240	test: 1.5953855	best: 1.5844244 (138)	total: 15m 10s	remaining: 10m 4s
700:	learn: 1.3171803	test: 1.5980556	best: 1.5844244 (138)	total: 17m 40s	remaining: 7m 32s
800:	learn: 1.3093925	test: 1.5997478	best: 1.5844244 (138)	total: 20m 10s	remaining: 5m
900:	learn: 1.3031041	test: 1.6012300	bes

# Feature Importance
Below we display the CatBoost top 25 feature importance for the last fold we trained.

feature importance 上位25件

In [ ]:
TOP = 25

feature_importance = model.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(10, 8))
plt.barh(np.arange(len(sorted_idx))[-TOP:], feature_importance[sorted_idx][-TOP:], align='center')
plt.yticks(np.arange(len(sorted_idx))[-TOP:], np.array(FEATURES)[sorted_idx][-TOP:])
plt.title(f'Feature Importance - Top {TOP}')
plt.show()

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

<Figure size 1000x800 with 0 Axes>

# CV Score for CatBoost
This is CV score for our CatBoost model.

In [ ]:
import sys
sys.path.append('../input/kaggle-kl-div')
from kaggle_kl_div import score

oof = pd.DataFrame(all_oof.copy())
oof['id'] = np.arange(len(oof))

true = pd.DataFrame(all_true.copy())
true['id'] = np.arange(len(true))

oof.head()

,0,1,2,3,4,5,id
0,0.282334,0.152483,0.119410,0.038550,0.082940,0.324284,0
1,0.107468,0.040065,0.012347,0.009851,0.027876,0.802393,1
2,0.098343,0.180791,0.040402,0.095698,0.138017,0.446748,2
3,0.105720,0.433510,0.083405,0.035412,0.034946,0.307007,3
4,0.199134,0.129931,0.079586,0.039890,0.160066,0.391394,4


In [ ]:
true.head()

,0,1,2,3,4,5,id
0,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0
1,0.000000,0.000000,0.0,0.0,0.000000,1.000000,1
2,0.066667,0.333333,0.0,0.2,0.066667,0.333333,2
3,1.000000,0.000000,0.0,0.0,0.000000,0.000000,3
4,0.000000,0.000000,0.0,0.0,1.000000,0.000000,4


In [ ]:
# TARGETSの各列を正規化
oof_data = oof[[0, 1, 2, 3, 4, 5]].values
oof_data = oof_data / oof_data.sum(axis=1,keepdims=True)
oof[[0, 1, 2, 3, 4, 5]] = oof_data

In [ ]:
oof.head()

,0,1,2,3,4,5,id
0,0.282334,0.152483,0.119410,0.038550,0.082940,0.324284,0
1,0.107468,0.040065,0.012347,0.009851,0.027876,0.802393,1
2,0.098343,0.180791,0.040402,0.095698,0.138017,0.446748,2
3,0.105720,0.433510,0.083405,0.035412,0.034946,0.307007,3
4,0.199134,0.129931,0.079586,0.039890,0.160066,0.391394,4


In [ ]:
temp = pl.from_pandas(oof)

In [ ]:
cv = score(solution=true, submission=oof, row_id_column_name='id')
print('CV Score KL-Div for CatBoost =',cv)

CV Score KL-Div for CatBoost = 1.1666203409357747


In [ ]:
import sys
sys.path.append('../input/kaggle-kl-div')
from kaggle_kl_div import score

oof = pd.DataFrame(all_oof.copy())
oof['id'] = np.arange(len(oof))

true = pd.DataFrame(all_true.copy())
true['id'] = np.arange(len(true))

cv = score(solution=true, submission=oof, row_id_column_name='id')
print('CV Score KL-Div for CatBoost =',cv)

# CV Score KL-Divの保存
with open(f'models/CV_Score_v{VER}.txt', 'w') as f:
    f.write(str(cv) + "\n")

CV Score KL-Div for CatBoost = 1.1666203409357747
